# Owning and Borrowing

Are concepts for safe memory management

## Owning

is the concept of the ownership over a memory reference (a value), normally indicated by a variable.


In [7]:
// Let's create a memory string (`Strings` are stored in the heap) and print it
let s = String::from("String");
println!("{}", s);

String


In [8]:
// we repeat the same steps and then continue at step 2
let s = String::from("String");
println!("{}", s);
// step 2: we point a new variable to be the new owner of that String in the heap
let new_string_owner = s;
println!("{}", new_string_owner);
// and this will fail. Check how the compiler tells us that the value "moved"
println!("{}", s);

Error: borrow of moved value: `s`

## Borrowing



In [ ]:
// we repeat the same steps and then continue at step 2
let mut s = String::from("String");
println!("{}", s);
// step 2: we point a new variable to be the new owner of that String in the heap
// but this time we use a reference
let new_string_owner = &s;
println!("{}", new_string_owner);
// and this will fail. Check how the compiler tells us that the value "moved"
println!("{}", s);